## Scrap https://law.judicial.gov.tw

In [1]:
import time
import numpy as np
import pandas as pd
from selenium import webdriver

browser = webdriver.Chrome('../../../chromedriver')

In [2]:
browser.get('https://law.judicial.gov.tw/FJUD/default.aspx')

#### Search by keyword '穿山甲'

In [3]:
search_input = browser.find_element_by_css_selector("input#txtKW")
search_input.send_keys('穿山甲')
search_submit_button = browser.find_element_by_css_selector("input#btnSimpleQry")
search_submit_button.click()

#### Collect ```title```, ```date```, ```cause of action```

In [4]:
def getTRows(_browser):
    time.sleep(2)
    _browser.switch_to.frame(browser.find_element_by_tag_name("iframe")) #switch to iframe
    trs = _browser.find_elements_by_css_selector("div.result-list table tbody tr")
    return trs
    
def getHrefFromRow(row):
    return row.find_element_by_css_selector('a').get_attribute('href')

def extractInfoFromRows(trows, _browser):
    list_of_dict = list()
    for index in range(1, len(trows)):
        #for each row
        if (index % 2 == 1):
            text_arr = trows[index].text.split()
            dist = {
                'court info': ' '.join(str(x) for x in text_arr[1:-2]),
                'date': text_arr[-2],
                'cause': text_arr[-1],
                'url': getHrefFromRow(trows[index])
            }
            list_of_dict.append(dist)
    return list_of_dict

def goNextPage(_browser):
    btn = _browser.find_element_by_css_selector('a#hlNext')
    btn.click()
    _browser.switch_to.default_content(); # backto root document
    return

In [5]:
trs1 = getTRows(browser)
pages = pd.DataFrame(extractInfoFromRows(trs1, browser))
pages.head()

,cause,court info,date,url
0,野生動物保育法,臺灣花蓮地方法院 108 年度 訴 字第 56 號刑事判決（10K）,108.03.25,https://law.judicial.gov.tw/FJUD/data.aspx?ro=...
1,野生動物保育法等,臺灣高等法院 107 年度 原上訴 字第 82 號刑事判決（13K）,107.10.03,https://law.judicial.gov.tw/FJUD/data.aspx?ro=...
2,毒品危害防制條例等,臺灣高等法院 臺南分院 107 年度 上訴 字第 478 號刑事判決（60K）,107.08.02,https://law.judicial.gov.tw/FJUD/data.aspx?ro=...
3,聲請撤銷緩刑,臺灣高等法院 高雄分院 107 年度 原抗 字第 1 號刑事裁定（5K）,107.07.16,https://law.judicial.gov.tw/FJUD/data.aspx?ro=...
4,違反野生動物保育法等,臺灣新竹地方法院 106 年度 原訴 字第 21 號刑事判決（16K）,107.04.27,https://law.judicial.gov.tw/FJUD/data.aspx?ro=...


### Page1 scraped, process the rest with pagination

In [7]:
page_num = 2
while (page_num < 7):
    time.sleep(1)
    goNextPage(browser)
    trs = getTRows(browser)
    page = pd.DataFrame(extractInfoFromRows(trs, browser))
    pages = pages.append(page, ignore_index=True)
    page_num = page_num + 1

In [8]:
pages

In [9]:
pages.to_csv('./court_cases_urls.csv', encoding='utf_8_sig')